# Offload and Idle Pull

This notebook demonstrates the workload behaviour for big core offload and idle pull

In [1]:
import logging
reload(logging)
logging.basicConfig(
    format='%(asctime)-9s %(levelname)-8s: %(message)s',
    datefmt='%I:%M:%S')

# Enable logging at INFO level
logging.getLogger().setLevel(logging.INFO)
# Comment the follwing line to disable devlib debugging statements
# logging.getLogger('ssh').setLevel(logging.DEBUG)

In [2]:
# Generate plots inline
%pylab inline

import json
import os
import subprocess

# Support to access the remote target
import devlib
from env import TestEnv

# Support for trace events analysis
from trace import Trace

# Support to configure and run RTApp based workloads
from wlgen import RTA

# Support for performance analysis of RTApp workloads
from perf_analysis import PerfAnalysis

# Suport for FTrace events parsing and visualization
import trappy

Populating the interactive namespace from numpy and matplotlib


# Test environment setup

In [3]:
# Setup a target configuration
my_target_conf = {
    
    "platform"    : 'android',
    "board"       : "hikey960",
    "device" : "0123456789ABCDEF",
    
    "ANDROID_HOME" : "/home/lisa/android-sdk",
    "rtapp-calib" : {"0": 302, "1": 302, "2": 304, "3": 304, "4": 136, "5": 137, "6": 136, "7": 136},

     "emeter" : {
        "instrument" : "acme",
        "conf" : {
            # Absolute path to the iio-capture binary on the host
            'iio-capture' : '/usr/bin/iio-capture',
            # Default host name of the BeagleBone Black
            'ip_address'     : '10.169.36.36',
        },
        "channel_map" : {
            "Device0" : 0, # iio:device0
            "Device1" : 1, # iio:device0
        }
    },

    # Define devlib module to load
    #"modules"     : [
    #    'bl',           # enable big.LITTLE support
    #    'cpufreq'       # enable CPUFreq support
    #],

    # Account to access the remote target
    "host"        : '10.169.36.25',
    "username"    : 'root',
    "password"    : '',



}

# Setup the required Test Environment supports
my_tests_conf = {
    
    # Binary tools required to run this experiment
    # These tools must be present in the tools/ folder for the architecture
    "tools"   : ['rt-app', 'taskset', 'trace-cmd'],
    
    # FTrace events end buffer configuration
    "ftrace"  : {
         "events" : [
            "cpu_frequency",
            "sched_load_avg_cpu",
            "sched_load_avg_task",
            "sched_switch",
            'dequeue_task_fair', 
            'enqueue_task_fair', 
            'set_next_entity',

                    
            "cpu_capacity",
            
            #/sys/kernel/debug/tracing/events/sched
            "sched_blocked_reason",
            "sched_boost_cpu",
            "sched_boost_task",
            "sched_contrib_scale_f",
            "sched_cpu_hotplug",
            "sched_energy_diff",
            "sched_kthread_stop",
            "sched_kthread_stop_ret",
            "sched_load_avg_cpu",
            "sched_load_avg_task",
            "sched_migrate_task",
            "sched_move_numa",
            "sched_pi_setprio",
            "sched_process_exec",
            "sched_process_exit",
            "sched_process_fork",
            "sched_process_free",
            "sched_process_wait",
            "sched_stat_blocked",
            "sched_stat_iowait",
            "sched_stat_runtime",
            "sched_stat_sleep",
            "sched_stat_wait",
            "sched_stick_numa",
            "sched_swap_numa",
            "sched_switch",
            "sched_tune_boostgroup_update",
            "sched_tune_config",
            "sched_tune_tasks_update",
            "sched_tune_filter",
            "sched_wait_task",
            "sched_wake_idle_without_ipi",
            "sched_wakeup",
            "sched_wakeup_new",
            
            
         ],
         "buffsize" : 10240
    },
    "results_dir" : "lisa_offload_and_idle_pull",
}

In [4]:
# Support to access the remote target
import devlib
from env import TestEnv

# Initialize a test environment using:
# the provided target configuration (my_target_conf)
# the provided test configuration   (my_test_conf)
te = TestEnv(target_conf=my_target_conf, test_conf=my_tests_conf)
target = te.target

06:47:38  INFO    : Using base path: /home/lisa/lisa-github
06:47:38  INFO    : Loading custom (inline) target configuration
06:47:38  INFO    : Loading custom (inline) test configuration
06:47:38  WARNING : Wipe previous contents of the results folder:
06:47:38  WARNING :    /home/lisa/lisa-github/results/lisa_offload_and_idle_pull
06:47:38  INFO    : External tools using:
06:47:38  INFO    :    ANDROID_HOME: /home/lisa/android-sdk
06:47:38  INFO    :    CATAPULT_HOME: /home/lisa/lisa-github/tools/catapult
06:47:38  INFO    : Devlib modules to load: ['bl', 'cpuidle', 'cpufreq']
06:47:38  INFO    : Connecting Android target [0123456789ABCDEF]
06:47:38  INFO    : Connection settings:
06:47:38  INFO    :    {'device': '0123456789ABCDEF'}
06:47:39  INFO    : Initializing target workdir:
06:47:39  INFO    :    /data/local/tmp/devlib-target
06:47:41  INFO    : Attempting to read energy model from target
06:47:44  INFO    : Topology:
06:47:44  INFO    :    [[0, 1, 2, 3], [4, 5, 6, 7]]
06:47:

# Workload configuration

In [5]:
# Support to configure and run RTApp based workloads
from wlgen import RTA, Periodic, Ramp

# Create a new RTApp workload generator using the calibration values
# reported by the TestEnv module
rtapp = RTA(target, 'simple', calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(
    # 1. generate a "profile based" set of tasks
    kind='profile',
    
    # 2. define the "profile" of each task
    params={
        
        "early_starter_1" : Periodic(
            period_ms=10,
            duty_cycle_pct=100,
            duration_s=5,
            cpus="4",
        ).get(),

        "early_starter_2" : Periodic(
            period_ms=10,
            duty_cycle_pct=100,
            duration_s=5,
            cpus="5",
        ).get(),

        "early_starter_3" : Periodic(
            period_ms=10,
            duty_cycle_pct=100,
            duration_s=5,
            cpus="6",
        ).get(),

        "early_starter_4" : Periodic(
            period_ms=10,
            duty_cycle_pct=100,
            duration_s=5,
            cpus="7",
        ).get(),
        
        "delay_starter_1" : Periodic(
            period_ms=10,
            duty_cycle_pct=100,
            duration_s=10,
            delay_s=1,
        ).get(),
        
        "delay_starter_2" : Periodic(
            period_ms=10,
            duty_cycle_pct=5,
            duration_s=10,
            delay_s=1,
        ).get(),

#        "delay_starter_3" : Periodic(
#            period_ms=10,
#            duty_cycle_pct=100,
#            duration_s=5,
#            delay_s=2,
#        ).get(),
#        
#        "delay_starter_4" : Periodic(
#            period_ms=10,
#            duty_cycle_pct=100,
#            duration_s=5,
#            delay_s=2,
#        ).get(),
    },
    
    # 4. use this folder for task logfiles
    run_dir=target.working_directory
    
);

06:47:47  INFO    : Setup new workload simple
06:47:47  INFO    : Workload duration defined by longest task
06:47:47  INFO    : Default policy: SCHED_OTHER
06:47:47  INFO    : ------------------------
06:47:47  INFO    : task [delay_starter_1], sched: using default policy
06:47:47  INFO    :  | start delay: 1.000000 [s]
06:47:47  INFO    :  | loops count: 1
06:47:47  INFO    :  + phase_000001: batch 10.000000 [s]
06:47:47  INFO    : ------------------------
06:47:47  INFO    : task [delay_starter_2], sched: using default policy
06:47:47  INFO    :  | start delay: 1.000000 [s]
06:47:47  INFO    :  | loops count: 1
06:47:47  INFO    : + phase_000001: duration 10.000000 [s] (1000 loops)
06:47:47  INFO    : |  period    10000 [us], duty_cycle   5 %
06:47:47  INFO    : |  run_time    500 [us], sleep_time   9500 [us]
06:47:47  INFO    : ------------------------
06:47:47  INFO    : task [early_starter_1], sched: using default policy
06:47:47  INFO    :  | loops count: 1
06:47:47  INFO    :  |

# Workload execution

In [6]:
# Set Sched governor
#
# For Juno big/LITTLE CPUs
#
logging.info("Target ABI: %s, CPus: %s",
             target.abi,
             target.cpuinfo.cpu_names)

target.cpufreq.set_all_governors('sched')

logging.info("Target current governor: %s",
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_governor")
            )
logging.info("Target big CPU max CPUfreq:%s\n\t\t    Target big CPU current CPUfreq: %s", 
             target.read_value("/sys/devices/system/cpu/cpu4/cpufreq/scaling_max_freq"),
             target.read_value("/sys/devices/system/cpu/cpu4/cpufreq/scaling_cur_freq")
             )

logging.info("Target LITTLE CPU max CPUfreq:%s \n\t\t    Target LITTLE CPU current CPUfreq: %s", 
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_max_freq"),
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_cur_freq")
             )

06:47:49  INFO    : Target ABI: arm64, CPus: ['A53', 'A53', 'A53', 'A53', 'A73', 'A73', 'A73', 'A73']
06:47:49  INFO    : Target current governor: sched
06:47:49  INFO    : Target big CPU max CPUfreq:2362000
		    Target big CPU current CPUfreq: 1805000
06:47:49  INFO    : Target LITTLE CPU max CPUfreq:1844000 
		    Target LITTLE CPU current CPUfreq: 1844000


In [7]:
logging.info('#### Setup FTrace')
te.ftrace.start()

logging.info('#### Start energy sampling')
te.emeter.reset()

logging.info('#### Start RTApp execution')
rtapp.run(out_dir=te.res_dir, cgroup="")

logging.info('#### Read energy consumption: %s/energy.json', te.res_dir)
nrg_report = te.emeter.report(out_dir=te.res_dir)

logging.info('#### Stop FTrace')
te.ftrace.stop()

trace_file = os.path.join(te.res_dir, 'trace.dat')
logging.info('#### Save FTrace: %s', trace_file)
te.ftrace.get_trace(trace_file)

logging.info('#### Save platform description: %s/platform.json', te.res_dir)
(plt, plt_file) = te.platform_dump(te.res_dir)

06:47:50  INFO    : #### Setup FTrace
06:47:52  INFO    : #### Start energy sampling


06:47:53  INFO    : #### Start RTApp execution
06:47:53  INFO    : Workload execution START:
06:47:53  INFO    :    /data/local/tmp/bin/rt-app /data/local/tmp/devlib-target/simple_00.json 2>&1
06:48:11  INFO    : #### Read energy consumption: /home/lisa/lisa-github/results/lisa_offload_and_idle_pull/energy.json
06:48:14  INFO    : #### Stop FTrace
06:48:15  INFO    : #### Save FTrace: /home/lisa/lisa-github/results/lisa_offload_and_idle_pull/trace.dat
06:48:16  INFO    : #### Save platform description: /home/lisa/lisa-github/results/lisa_offload_and_idle_pull/platform.json


# Collected results

In [8]:
# All data are produced in the output folder defined by the TestEnv module
logging.info('Content of the output folder %s', te.res_dir)
!ls -la {te.res_dir}

06:48:16  INFO    : Content of the output folder /home/lisa/lisa-github/results/lisa_offload_and_idle_pull


total 15516
drwxrwxr-x 1 lisa lisa      560 Sep  6 06:48 .
drwxrwxr-x 1 lisa lisa     2606 Sep  6 06:47 ..
-rw-rw-r-- 1 lisa lisa       43 Sep  6 06:48 energy.json
-rw-rw-r-- 1 lisa lisa      359 Sep  6 06:48 energy_stats.json
-rw-rw-r-- 1 lisa lisa      855 Sep  6 06:48 output.log
-rw-rw-r-- 1 lisa lisa     1518 Sep  6 06:48 platform.json
-rw-r--r-- 1 lisa lisa      284 Sep  6 06:48 rt-app-delay_starter_1-0.log
-rw-r--r-- 1 lisa lisa   124160 Sep  6 06:48 rt-app-delay_starter_2-1.log
-rw-r--r-- 1 lisa lisa      284 Sep  6 06:48 rt-app-early_starter_1-2.log
-rw-r--r-- 1 lisa lisa      284 Sep  6 06:48 rt-app-early_starter_2-3.log
-rw-r--r-- 1 lisa lisa      284 Sep  6 06:48 rt-app-early_starter_3-4.log
-rw-r--r-- 1 lisa lisa      284 Sep  6 06:48 rt-app-early_starter_4-5.log
-rw-rw-r-- 1 lisa lisa    67597 Sep  6 06:48 samples_Device0.csv
-rw-rw-r-- 1 lisa lisa    60236 Sep  6 06:48 samples_Device1.csv
-rw-r--r-- 1 lisa lisa     2119 Sep  6 06:48 simple_00.json
-rw-r--r-- 1 lisa lisa 1

In [9]:
# Inspect the JSON file used to run the application
with open('{}/simple_00.json'.format(te.res_dir), 'r') as fh:
    rtapp_json = json.load(fh, )
logging.info('Generated RTApp JSON file:')
#print json.dumps(rtapp_json, indent=4, sort_keys=True)

06:48:18  INFO    : Generated RTApp JSON file:


In [10]:
# Dump the energy measured for the LITTLE and big clusters
logging.info('Energy: %s', nrg_report.report_file)
print json.dumps(nrg_report.channels, indent=4, sort_keys=True)

06:48:19  INFO    : Energy: /home/lisa/lisa-github/results/lisa_offload_and_idle_pull/energy.json


{
    "Device0": 0.0, 
    "Device1": 0.0
}


In [11]:
# Dump the platform descriptor, which could be useful for further analysis
# of the generated results
logging.info('Platform description: %s', plt_file)
#print json.dumps(plt, indent=4, sort_keys=True)

06:48:19  INFO    : Platform description: /home/lisa/lisa-github/results/lisa_offload_and_idle_pull/platform.json


# Trace inspection

In [12]:
# Suport for FTrace events parsing and visualization
import trappy

# NOTE: The interactive trace visualization is available only if you run
#       the workload to generate a new trace-file
trappy.plotter.plot_trace(te.res_dir)

# New Task Behavior

In [13]:
events_to_parse = my_tests_conf['ftrace']['events']

trace = Trace(plt, te.res_dir, events_to_parse)

ftrace = trace.ftrace

trappy.plotter.plot_trace(ftrace, execnames=['early_starter_1', "early_starter_2","early_starter_3", "early_starter_4", "delay_starter_1",
                                             "delay_starter_2", "delay_starter_3", "delay_starter_4",
                                            ])

/home/lisa/lisa-github/libs/trappy/trappy/ftrace.py:205: UserWarning: TRAPpy: Couldn't read <trappy.dynamic.set_next_entity object at 0x7f7106673090> from cache, reading it from trace
  warnings.warn(warnstr)
/home/lisa/lisa-github/libs/trappy/trappy/ftrace.py:205: UserWarning: TRAPpy: Couldn't read <trappy.dynamic.cpu_frequency_devlib object at 0x7f71066731d0> from cache, reading it from trace
  warnings.warn(warnstr)
/home/lisa/lisa-github/libs/trappy/trappy/ftrace.py:205: UserWarning: TRAPpy: Couldn't read <trappy.dynamic.sched_move_numa object at 0x7f71066732d0> from cache, reading it from trace
  warnings.warn(warnstr)
/home/lisa/lisa-github/libs/trappy/trappy/ftrace.py:205: UserWarning: TRAPpy: Couldn't read <trappy.dynamic.sched_wait_task object at 0x7f71066733d0> from cache, reading it from trace
  warnings.warn(warnstr)
/home/lisa/lisa-github/libs/trappy/trappy/ftrace.py:205: UserWarning: TRAPpy: Couldn't read <trappy.dynamic.sched_stat_blocked object at 0x7f71066735d0> from c

# Trace Overview

In [16]:
# NOTE: The interactive trace visualization is available only if you run
#       the workload to generate a new trace-file
trappy.plotter.plot_trace(ftrace)